# Enrichment NB 05: Analyzing rSEA results

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
import altair as alt
from toolz.curried import pipe

In [8]:
ALPHA = 0.05
MSV = 0.5

plots_dir = "altair_data"

go_results_file = os.path.join("results_04_rSEA", "enrichment_rsea_thresh_0.99_lib_GO_Biological_Process_2018.tsv")
reactome_results_file = os.path.join("results_04_rSEA", "enrichment_rsea_thresh_0.99_lib_ReactomePathways.tsv")

diff_expr_file = os.path.join("diff_expr", "all_results.tsv.gz")

gene_set_dir = "gene_set_libraries"
go_gmt = os.path.join(gene_set_dir, "GO_Biological_Process_2018.gmt")
reactome_gmt = os.path.join(gene_set_dir, "ReactomePathways.gmt")

In [2]:
results_files = {
    "go": go_results_file,
    "reactome": reactome_results_file
}

results = {}

for gene_set in results_files.keys():
    
    result = pd.read_csv(results_files[gene_set], sep="\t", index_col=0)
    result = result[
        (result["Coverage"] > 0) & 
        (
            (result["SC.adjP"] <= ALPHA) | 
            (result["Comp.0.99.adjP"] <= ALPHA)
        )]
    
    grp_split = result["cancer_type_group"].str.split("_", n=1, expand=True)
    
    result = result.assign(
        cancer_type=grp_split[0],
        group=grp_split[1]
    ).\
    drop(columns="cancer_type_group").\
    sort_values(
        by=["cancer_type", "group", "SC.adjP", "Coverage", "Size"],
        ascending=[True, True, True, False, False]
    )
    
    results[gene_set] = result

In [3]:
# Altair options
alt.data_transformers.disable_max_rows()

def json_dir(data, data_dir):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=os.path.join(data_dir, "{prefix}-{hash}.{extension}")) )

alt.data_transformers.register("json_dir", json_dir)
alt.data_transformers.enable("json_dir", data_dir="altair_data")

DataTransformerRegistry.enable('json_dir')

## Distribution of proportion of gene sets covered

In [4]:
alt.Chart(results["go"]).mark_bar().encode(
    x=alt.X(
        "Coverage",
        bin=alt.Bin(step=0.01),
        scale=alt.Scale(domain=[0, 1])
    ),
    y=alt.Y(
        "count()"
    )
)

alt.Chart(...)

In [5]:
# Due to there being a duplicated gene name, and a gene set comprised only of that gene,
# there is one entry that's marked as having a coverage proportion of 2. So that doesn't
# mess up our chart, we'll just clip it out.

alt.Chart(results["reactome"]).mark_bar(clip=True).encode(
    x=alt.X(
        "Coverage",
        bin=alt.Bin(step=0.01),
        scale=alt.Scale(domain=[0, 1])
    ),
    y=alt.Y(
        "count()"
    )
)

alt.Chart(...)

## Look at individual groups

In [6]:
results["go"].groupby(["cancer_type", "group"]).head().set_index(["cancer_type", "group", "Name"])

Size  \
cancer_type group        Name                                                       
colon       with_both    protein export from nucleus (GO:0006611)              29   
                         intracellular protein transport (GO:0006886)         348   
                         nuclear export (GO:0051168)                           93   
                         mitotic nuclear envelope reassembly (GO:0007084)      11   
                         peptidyl-serine dephosphorylation (GO:0070262)        11   
            with_gain    Golgi vesicle budding (GO:0048194)                    10   
                         positive regulation of viral genome replication...    32   
                         regulation of viral genome replication (GO:0045...    64   
                         positive regulation of viral life cycle (GO:190...    45   
                         modification by symbiont of host morphology or ...     8   
            with_loss    intracellular protein transport (GO:0006886)         348   
                         protein export from nucleus (GO:0006611)              29   
                         nuclear export (GO:0051168)                           93   
                         positive regulation of exocytosis (GO:0045921)        35   
                         positive regulation of secretion by cell (GO:19...    53   
            without_both snRNA transcription (GO:0009301)                      71   
                         snRNA transcription from RNA polymerase II prom...    71   
                         transcription elongation from RNA polymerase II...    96   
                         transcription initiation from RNA polymerase II...   161   
                         DNA-templated transcription, elongation (GO:000...   109   
            without_gain establishment of mitochondrion localization, mi...     7   
                         positive regulation of mitochondrial membrane p...     9   
                         regulation of mitochondrion organization (GO:00...    49   
                         mitochondrion organization (GO:0007005)              168   
                         organelle transport along microtubule (GO:0072384)    32   
            without_loss intracellular lipid transport (GO:0032365)            16   
                         fatty acid transmembrane transport (GO:1902001)       12   
                         carnitine shuttle (GO:0006853)                        12   
                         long-chain fatty acid transport (GO:0015909)          26   
                         fatty acid metabolic process (GO:0006631)            107   
...                                                                           ...   
ovarian     with_both    purine nucleoside monophosphate biosynthetic pr...    13   
                         ribonucleoside monophosphate biosynthetic proce...    15   
                         purine ribonucleoside monophosphate biosyntheti...    40   
                         purine ribonucleoside monophosphate metabolic p...    40   
                         hemidesmosome assembly (GO:0031581)                   12   
            with_gain    coenzyme A metabolic process (GO:0015936)             11   
                         ribonucleoside bisphosphate biosynthetic proces...     8   
                         purine nucleoside bisphosphate biosynthetic pro...     8   
                         coenzyme A biosynthetic process (GO:0015937)           8   
                         coenzyme biosynthetic process (GO:0009108)            30   
            with_loss    regulation of response to cytokine stimulus (GO...    21   
                         positive regulation of response to cytokine sti...    15   
                         regulation of interferon-alpha secretion (GO:19...     7   
                         positive regulation of interferon-alpha secreti...     7   
                         positive regulation of tumor necrosis factor se...    12 

## Let's try a bubble plot

In [9]:
pd.read_csv(diff_expr_file, sep="\t")

,group,cancer_type,protein,change,p_val,t_stat,protein_str,adj_p
0,with_gain,colon,A1BG,-1.395839,4.718120e-20,-12.250373,A1BG,6.593974e-19
1,with_gain,colon,A1CF,0.030382,7.333480e-01,0.342409,A1CF,7.798772e-01
2,with_gain,colon,A2M,-1.407581,2.342255e-24,-13.665237,A2M,5.114135e-23
3,with_gain,colon,AAAS,0.284686,6.883279e-06,5.014562,AAAS,1.936078e-05
4,with_gain,colon,AACS,0.245039,5.258337e-04,3.632179,AACS,1.126254e-03
5,with_gain,colon,AAGAB,0.229421,1.318995e-04,4.112841,AAGAB,3.091836e-04
6,with_gain,colon,AAK1,-0.284165,6.063943e-10,-7.266965,AAK1,2.838273e-09
7,with_gain,colon,AAMDC,-0.769043,2.400923e-15,-9.157098,AAMDC,2.049528e-14
8,with_gain,colon,AAMP,0.105842,4.298724e-02,2.073340,AAMP,6.519022e-02
9,with_gain,colon,AAR2,0.174863,7.646303e-04,3.496326,AAR2,1.596878e-03


In [ ]:
def plot_top_ten(
    enrich_file_path, 
    expr_file_path, 
    gmt_file_path, 
    xtitle, 
    min_size, 
    sort_col, 
    sort_asc, 
    cutoff_col, 
    cutoff=0.05
):
    
    # Read in the expression data
    all_expression_data = pd.read_csv(expr_file_path, sep="\t", index_col=0)

    # Make a column where all increases are +1 and all decreases 
    # are -1, because these are ratioed abundances, so we can't 
    # compare magnitudes between genes--we can only compare whether 
    # there was a change, and whether it was positive or negative
    all_expression_data = all_expression_data.assign(simplified_change=np.nan)

    # adj p < 0.05 and change > 1 => +1
    all_expression_data.loc[
        (all_expression_data["change"] > 0) & (all_expression_data["adj_p"] < 0.05), 
        "simplified_change"
    ] = 1

    # adj p >= 0.05 and change > 1 => +0.5
    all_expression_data.loc[(all_expression_data["change"] > 0) & (all_expression_data["adj_p"] >= 0.05),
        "simplified_change"
    ] = MSV

    # change == 0 => 0
    all_expression_data.loc[
        all_expression_data["change"] == 0,
        "simplified_change"
    ] = 0

    # adj p >= 0.05 and change < 1 => -0.5
    all_expression_data.loc[(all_expression_data["change"] < 0) & (all_expression_data["adj_p"] >= 0.05), 
        "simplified_change"
    ] = -MSV

    # adj p < 0.05 and change < 1 => -1
    all_expression_data.loc[
        (all_expression_data["change"] < 0) & (all_expression_data["adj_p"] < 0.05),
        "simplified_change"
    ] = -1

    # Select just the proteins where we chose to reject the null hypothesis of no change
    # We'll use this later to calculate average change in expression for each pathway
    expression_data = all_expression_data[all_expression_data["adj_p"] <= 0.05]
    
    # Read in the GMT file and join in pathway names and gene lists
    with open(gmt_file_path, "r") as fp:
        gene_lists = fp.readlines()

    # Take the newline off the end, and split on the tab character to create a list of lists
    gene_lists = [l.strip().split("\t") for l in gene_lists]

    # Create a dataframe mapping pathway ID to pathway name and genes
    pathway_names = [l[0] for l in gene_lists]
    pathway_genes = [l[2:] for l in gene_lists] # We skip index 1 -- it's a blank unused field.
    pathway_data = pd.DataFrame({
        "pathway_name": pathway_names,
        "pathway_genes": pathway_genes
    })

    # Read in the enrichment data
    enrichment_data = pd.read_csv(enrich_file_path, sep="\t", index_col=0)
    enrichment_data.columns = enrichment_data.columns.str.replace(".", "_", regex=False)
    enrichment_data = enrichment_data.rename(columns={"Name": "pathway_name"})
    
    # Merge pathway data into the enrichment data
    enrichment_data = enrichment_data.merge(
        pathway_data,
        how="left",
        left_on="pathway_name",
        right_on="pathway_name",
        validate="many_to_one"
    )
    
    # Select pathways that meet the minimum size
    enrichment_data = enrichment_data[
        enrichment_data["pathway_genes"].apply(len) >= min_size
    ]

    # Select enrichment data below our cutoff
    enrichment_data = enrichment_data.loc[enrichment_data[cutoff_col] <= cutoff, :]
    
    # Assign pathway ranks within each cancer type based on the sort_col. Make a custom one if needed.
    if sort_col != cutoff_col:
        if sort_asc:
            enrichment_data = enrichment_data.assign(
                temp_sort=(enrichment_data[sort_col] + 0.1) * (enrichment_data[cutoff_col] + 0.1)
            )
        else:
            enrichment_data = enrichment_data.assign(
                temp_sort=(enrichment_data[sort_col] + 0.1) / (enrichment_data[cutoff_col] + 0.1)
            )
            
        sort_col = "temp_sort"
    
    enrichment_data = enrichment_data.\
        assign(cancer_rank=enrichment_data.groupby("cancer_type")[sort_col].rank(ascending=sort_asc)).\
        sort_values(by=["cancer_type", "cancer_rank"]).\
        reset_index(drop=True)

    # Make a table with summary info for all pathways
    enrichment_summary = enrichment_data[["pathway_name"]].drop_duplicates(keep="first")

    pathway_times_enriched = enrichment_summary["pathway_name"].apply(
        lambda x: enrichment_data[enrichment_data["pathway_name"] == x].shape[0])

    avg_rank = enrichment_summary["pathway_name"].apply(
        lambda x: enrichment_data.loc[enrichment_data["pathway_name"] == x, "cancer_rank"].mean())

    enrichment_summary = enrichment_summary.\
        assign(
            pathway_times_enriched=pathway_times_enriched,
            pathway_avg_rank=avg_rank).\
        sort_values(
            by=["pathway_times_enriched", "pathway_avg_rank"],
            ascending=[False, True]).\
        reset_index(drop=True)

    # Merge in the original enrichment data
    enrichment_data = enrichment_data.\
        merge(
            enrichment_summary,
            how="outer",
            left_on="pathway_name",
            right_on="pathway_name",
            validate="many_to_one"
        ).\
        sort_values(
            by=["pathway_times_enriched", "pathway_avg_rank", "cancer_type"],
            ascending=[False, True, True]
        )

    # Select top 10 for our plot
    in_all = enrichment_summary.loc[
        enrichment_summary["pathway_times_enriched"] == 8,
        "pathway_name"
    ]
    
    if in_all.size <= 10:
        top_ten = in_all
    else:
        top_ten = in_all[:10]
    
    sel_enrich = enrichment_data[enrichment_data["pathway_name"].isin(top_ten)]

    # Calculate the mean expression for each pathway in each cancer type
    mean_exprs = []

    for idx in sel_enrich.index:
        genes = sel_enrich.loc[idx, "pathway_genes"]
        cancer_type = sel_enrich.loc[idx, "cancer_type"]

        genes_expr = expression_data.loc[
            expression_data["protein_str"].isin(genes),
            "simplified_change"
        ].mean()

        mean_exprs.append(genes_expr)

    sel_enrich = sel_enrich.assign(mean_expr=mean_exprs)

    sel_enrich = sel_enrich.assign(
        rank_size=1 / sel_enrich["cancer_rank"],
        avg_rank_size=1 / sel_enrich["pathway_avg_rank"],
        avg_rank_label=sel_enrich["pathway_avg_rank"].apply(round, 2).astype(str))
    
    # Take care of duplicates for the upper plot
    for i in range(10):
        sel_enrich["avg_rank_label"] = sel_enrich["avg_rank_label"].where(
            cond=~(sel_enrich.duplicated(subset=["cancer_type", "avg_rank_label"], keep="first")),
            other=" " + sel_enrich["avg_rank_label"])

    individual = alt.Chart(sel_enrich).mark_circle().encode(
        x=alt.X(
            "pathway_name:N",
            sort=sel_enrich["pathway_name"].values,
            axis=alt.Axis(
                labelAngle=-30,
                labelFontSize=12,
                labelLimit=500,
                title="",
                titleFontSize=16
            )
        ),
        y=alt.Y(
            "cancer_type:N",
            axis=alt.Axis(
                title="Cancer type",
                titleFontSize=12
            ),
        ),
        size=alt.Size(
            "rank_size:Q",
            legend=None
        ),
        color=alt.Color(
            "mean_expr:Q",
            scale=alt.Scale(
                scheme="blueorange",
                domain=[-1, 1]
            ),
            legend=alt.Legend(
                title="Pathway tumor expression"
            )
        )
    ).properties(
        width=400,
        height=300
    )

    aggregate = alt.Chart(sel_enrich).mark_circle().encode(
        x=alt.X(
            "avg_rank_label:N",
            sort=sel_enrich["avg_rank_label"].values,
            axis=alt.Axis(
                labelAngle=-30,
                labelFontSize=12,
                labelLimit=500,
                title="Overall rank of pathway",
                titleFontSize=12
            )
        ),
        size=alt.Size(
            "avg_rank_size:Q",
            legend=None
        ),
    ).properties(
        width=400
    )

    full_plot = alt.vconcat(
        aggregate, individual
    ).properties(
        title=xtitle
    )
    
    return full_plot, enrichment_data, all_expression_data, enrichment_summary

In [ ]:
plot_top_ten(
    enrich_file_path=go_results_file, 
    expr_file_path=diff_expr_file, 
    gmt_file_path=go_gmt,
    xtitle="Reactome data - rSEA, threshold = 1, sort by Coverage / Comp_adjP, cutoff Comp_adjP",
    min_size=5,
    sort_col="Coverage",
    sort_asc=False,
    cutoff_col="Comp_adjP",
    cutoff=0.05
)[0],